<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/AHP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import numpy as np
import pandas as pd
from copy import deepcopy as cp

In [175]:
from pandas.core.indexes.numeric import Int64Index
class AHP:
  def __init__(self, choices, criteria):
    self.choices = choices
    self.criteria = criteria
    self.pairwise_matrix = np.ones(shape = (len(self.criteria), len(self.criteria)))
    self.index_criteria = {index:criteria for index, criteria in enumerate(self.criteria)}
    self.representation = [str(k) + '. ' + v for k, v in self.index_criteria.items()]
    self.ratings = [i for i in range(1, 10)]

  def Pairwise1(self):
    pairs = []
    pair_matrix = {}
    for index, criteria in enumerate(self.criteria):
      print('comparison')
      print('_'*60)

      for i in range(index+1, len(self.criteria)):
        print()
        print(f'criteria'.upper())
        print('\n'.join(self.representation))
        print()
        choice = int(input(f'{criteria} - {self.criteria[i]} :'))
        rating = int(input('rating out of 9 : '))
        # UPDATE pairwise_matrix
        arrangement = []
        if i != index:
          if choice != index:
            self.pairwise_matrix[choice][index] = rating
            self.pairwise_matrix[index][choice] = 1 / rating
          else:
            self.pairwise_matrix[choice][i] = rating
            self.pairwise_matrix[i][choice] = 1 / rating
            print(f"CURRENT MATRIX: {self.pairwise_matrix}")
          print()
          pairs.append(criteria + "-" + self.criteria[i])
    self.pairwise_matrix = np.round(self.pairwise_matrix, decimals = 3)
    return (self.pairwise_matrix, pd.DataFrame(self.pairwise_matrix, columns = self.criteria, index = self.criteria))

In [176]:
analytica = AHP(choices = ['Job A', 'Job B', 'Job C'],
                criteria = ['Benefits', 'Advancement', 'Location', 'Training'])
# print(analytica.pairwise_matrix)
_,original_df = analytica.Pairwise1()
# print(analytica.pairwise_matrix)

comparison
____________________________________________________________

CRITERIA
0. price
1. mpg
2. comfort
3. style

price - mpg :0
rating out of 9 : 3
CURRENT MATRIX: [[1.         3.         1.         1.        ]
 [0.33333333 1.         1.         1.        ]
 [1.         1.         1.         1.        ]
 [1.         1.         1.         1.        ]]


CRITERIA
0. price
1. mpg
2. comfort
3. style

price - comfort :0
rating out of 9 : 2
CURRENT MATRIX: [[1.         3.         2.         1.        ]
 [0.33333333 1.         1.         1.        ]
 [0.5        1.         1.         1.        ]
 [1.         1.         1.         1.        ]]


CRITERIA
0. price
1. mpg
2. comfort
3. style

price - style :0
rating out of 9 : 2
CURRENT MATRIX: [[1.         3.         2.         2.        ]
 [0.33333333 1.         1.         1.        ]
 [0.5        1.         1.         1.        ]
 [0.5        1.         1.         1.        ]]

comparison
_______________________________________________

#synthesis

In [177]:
copy_df = original_df.copy(deep=True)
summation = copy_df.sum()
print(summation)

price       2.333
mpg        12.000
comfort     5.250
style       3.750
dtype: float64


In [178]:
copy_df

,price,mpg,comfort,style
price,1.000,3.0,2.00,2.00
mpg,0.333,1.0,0.25,0.25
comfort,0.500,4.0,1.00,0.50
style,0.500,4.0,2.00,1.00


In [179]:
priority_vector = []
cols = list(original_df.columns)
for row_index in range(len(copy_df)):
  for i in range(len(cols)):
    copy_df.iloc[row_index][i] = round(copy_df.iloc[row_index, i] / summation[cols[i]], 3)
  priority_vector.append(round(copy_df.iloc[row_index].sum()/len(cols), 3))
print(priority_vector)


[0.398, 0.085, 0.218, 0.299]


In [180]:
copy_df['priority_vector'] = priority_vector

In [181]:
copy_df

,price,mpg,comfort,style,priority_vector
price,0.429,0.250,0.381,0.533,0.398
mpg,0.143,0.083,0.048,0.067,0.085
comfort,0.214,0.333,0.190,0.133,0.218
style,0.214,0.333,0.381,0.267,0.299


#consinstency ratio

In [183]:
consistency = np.round(np.dot(original_df.to_numpy(), np.array(priority_vector)),
                        decimals = 3)
print(consistency)

[1.687 0.347 0.907 1.274]


### divide it with priority vector

In [185]:
lambda_max = round((consistency / np.array(priority_vector)).mean(), 3)
print(lambda_max)

4.186


consistency index

In [187]:
C_I = round((lambda_max - len(analytica.criteria)) / (len(analytica.criteria) - 1),
            3)
print(C_I)

0.062


In [202]:
all_rgb = [0,0,0,0.58, 0.90, 1.12, 1.24, 1.32, 1.41]
r_i = {i:all_rgb[i] for i in range(3, 9)}
C_ratio = round(C_I / r_i[len(analytica.criteria)], 3)

In [203]:
C_ratio

0.069

# Choices pairwise matrix

In [204]:
choice_pairwise2 = []
choices_pairwise = AHP(choices=['Job A', 'Job B', 'Job C'],
                       criteria=['Job A', 'Job B', 'Job C'])

for criteria in analytica.criteria:
    print(f"CRITERIA: {criteria.upper()}")
    print("*" * 50)
    arr, original_df2 = choices_pairwise.Pairwise1()
    choice_pairwise2.append(original_df2.copy())  # Make sure to append a copy of arr
    # print("FAKE: ", arr)

# Now choice_pairwise2 should contain distinct arrays.


CRITERIA: PRICE
**************************************************
comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Saturn :1
rating out of 9 : 3


CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Cavalier :2
rating out of 9 : 4

comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Saturn - Cavalier :2
rating out of 9 : 2

comparison
____________________________________________________________
CRITERIA: MPG
**************************************************
comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Saturn :1
rating out of 9 : 4


CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Cavalier :2
rating out of 9 : 6

comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Saturn - Cavalier :2
rating out of 9 : 3

co

In [205]:
choice_pairwise2

[          Accord  Saturn  Cavalier
 Accord       1.0   0.333      0.25
 Saturn       3.0   1.000      0.50
 Cavalier     4.0   2.000      1.00,
           Accord  Saturn  Cavalier
 Accord       1.0    0.25     0.167
 Saturn       4.0    1.00     0.333
 Cavalier     6.0    3.00     1.000,
           Accord  Saturn  Cavalier
 Accord     1.000   2.000       8.0
 Saturn     0.500   1.000       6.0
 Cavalier   0.125   0.167       1.0,
           Accord  Saturn  Cavalier
 Accord      1.00   0.333       4.0
 Saturn      3.00   1.000       7.0
 Cavalier    0.25   0.143       1.0]

# choice synthesis

In [206]:
pair_copy = cp(choice_pairwise2)
print(f"ALL DF: {pair_copy}")
overall_priorities = []
for choiceDf in pair_copy:
  choice_df = choiceDf.copy(deep=True)
  summation = choice_df.sum()
  # print(summation)

  priority_v = []
  cols = list(choiceDf.columns)
  for row_index in range(len(choice_df)):
    for i in range(len(cols)):
      choice_df.iloc[row_index][i] = round(choice_df.iloc[row_index, i] / summation[cols[i]], 3)
    priority_v.append(round(choice_df.iloc[row_index].sum()/len(cols), 3))
  print(f"prio: {priority_v}")
  print(f"df: {choiceDf.head()}")
  # APPEND TO OVERALL

  overall_priorities.append(priority_v)

  # print(overall_prio_df)

ALL DF: [          Accord  Saturn  Cavalier
Accord       1.0   0.333      0.25
Saturn       3.0   1.000      0.50
Cavalier     4.0   2.000      1.00,           Accord  Saturn  Cavalier
Accord       1.0    0.25     0.167
Saturn       4.0    1.00     0.333
Cavalier     6.0    3.00     1.000,           Accord  Saturn  Cavalier
Accord     1.000   2.000       8.0
Saturn     0.500   1.000       6.0
Cavalier   0.125   0.167       1.0,           Accord  Saturn  Cavalier
Accord      1.00   0.333       4.0
Saturn      3.00   1.000       7.0
Cavalier    0.25   0.143       1.0]
prio: [0.123, 0.32, 0.557]
df:           Accord  Saturn  Cavalier
Accord       1.0   0.333      0.25
Saturn       3.0   1.000      0.50
Cavalier     4.0   2.000      1.00
prio: [0.087, 0.274, 0.639]
df:           Accord  Saturn  Cavalier
Accord       1.0    0.25     0.167
Saturn       4.0    1.00     0.333
Cavalier     6.0    3.00     1.000
prio: [0.593, 0.341, 0.066]
df:           Accord  Saturn  Cavalier
Accord     1.000 

In [207]:
overall_prio_df = pd.DataFrame(np.array(overall_priorities).transpose())
overall_prio_df

,0,1,2,3
0,0.123,0.087,0.593,0.265
1,0.320,0.274,0.341,0.656
2,0.557,0.639,0.066,0.080


In [209]:
composite_priority = np.round(np.dot(overall_prio_df.to_numpy(),
                            np.array(priority_vector).transpose()),
                              decimals = 3)
composite_priority

array([0.265, 0.421, 0.314])